In [82]:
import numpy as np
import os, tempfile
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

import tensorflow as tf
from tensorflow.keras import models, layers
from tensorflow.keras.preprocessing.image import load_img

from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

from sklearn.dummy import DummyClassifier
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from sklearn.dummy import DummyRegressor
from sklearn.dummy import DummyClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import roc_auc_score, plot_roc_curve
from sklearn.tree import export_graphviz
from sklearn.tree import DecisionTreeClassifier
import sklearn.metrics as metrics
from scipy import stats as stats
import statsmodels.api as sm
from statsmodels.formula.api import ols
%matplotlib inline
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split

In [83]:
pd17 = pd.read_csv("./data/2017nfldata.csv")
pd18 = pd.read_csv("./data/2018nfldata.csv")
pd19 = pd.read_csv("./data/2019nfldata.csv")
pd20 = pd.read_csv("./data/2020nfldata.csv")
pd21 = pd.read_csv("./data/2021nfldata.csv")

In [84]:
pd17.head()

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Todd Gurley*+,LAR,RB,23,15,15,0,0,0,...,NaN,NaN,319.0,383.3,391.3,351.3,192.0,1,1.0,GurlTo01
1,2,Le'Veon Bell*+,PIT,RB,25,15,15,0,0,0,...,NaN,NaN,257.0,341.6,349.6,299.1,130.0,2,2.0,BellLe00
2,3,Kareem Hunt*,KAN,RB,22,16,16,0,0,0,...,NaN,NaN,242.0,295.2,302.2,268.7,115.0,3,3.0,HuntKa00
3,4,Alvin Kamara*,NOR,RB,22,16,3,0,0,0,...,1.0,NaN,239.0,320.4,327.4,279.9,112.0,4,4.0,KamaAl00
4,5,Melvin Gordon,LAC,RB,24,16,16,0,0,0,...,NaN,NaN,230.0,288.1,294.1,259.1,103.0,5,5.0,GordMe00


In [85]:
pd18.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Todd Gurley*+,LAR,RB,24,14,14,0,0,0,...,3.0,NaN,313.0,372.1,379.1,342.6,178.0,1,1.0,GurlTo01
1,2,Saquon Barkley*,NYG,RB,21,16,16,0,0,0,...,1.0,NaN,295.0,385.8,391.8,340.3,159.0,2,2.0,BarkSa00
2,3,Christian McCaffrey,CAR,RB,22,16,16,1,1,50,...,NaN,NaN,279.0,385.5,392.5,332.0,143.0,3,3.0,McCaCh01


In [86]:
pd19.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Christian McCaffrey*+,CAR,RB,23,16,16,0,2,0,...,1.0,NaN,355.0,471.2,477.2,413.2,215.0,1,1.0,McCaCh01
1,2,Lamar Jackson*+,BAL,QB,22,15,15,265,401,3127,...,NaN,NaN,416.0,415.7,429.7,421.7,152.0,1,2.0,JackLa00
2,3,Derrick Henry *,TEN,RB,25,15,15,0,0,0,...,NaN,NaN,277.0,294.6,303.6,285.6,136.0,2,3.0,HenrDe00


In [87]:
pd20.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Derrick Henry *+,TEN,RB,26,16,16,0,0,0,...,1.0,NaN,314.0,333.1,341.1,323.6,184.0,1,1.0,HenrDe00
1,2,Alvin Kamara*,NOR,RB,25,15,10,0,0,0,...,NaN,NaN,295.0,377.8,383.8,336.3,165.0,2,2.0,KamaAl00
2,3,Dalvin Cook*,MIN,RB,25,14,14,0,0,0,...,3.0,NaN,294.0,337.8,346.8,315.8,164.0,3,3.0,CookDa01


In [88]:
pd21.head(3)

,Rk,Player,Tm,FantPos,Age,G,GS,Cmp,Att,Yds,...,2PM,2PP,FantPt,PPR,DKPt,FDPt,VBD,PosRank,OvRank,-9999
0,1,Jonathan Taylor*+,IND,RB,22,17,17,0.0,0.0,0.0,...,NaN,NaN,333.0,373.1,381.1,353.1,187.0,1,1.0,TaylJo02
1,2,Cooper Kupp*+,LAR,WR,28,17,17,0.0,1.0,0.0,...,1.0,NaN,295.0,439.5,442.5,367.0,173.0,1,2.0,KuppCo00
2,3,Deebo Samuel*+,SFO,WR,25,16,15,1.0,2.0,24.0,...,NaN,NaN,262.0,339.0,347.0,300.5,140.0,2,3.0,SamuDe00


In [89]:
for col in pd17.columns:
    print(col)

Rk
Player
Tm
FantPos
Age
G
GS
Cmp
Att
Yds
TD
Int
Att.1
Yds.1
Y/A
TD.1
Tgt
Rec
Yds.2
Y/R
TD.2
Fmb
FL
TD.3
2PM
2PP
FantPt
PPR
DKPt
FDPt
VBD
PosRank
OvRank
-9999


In [90]:
for col in pd18.columns:
    print(col)

Rk
Player
Tm
FantPos
Age
G
GS
Cmp
Att
Yds
TD
Int
Att.1
Yds.1
Y/A
TD.1
Tgt
Rec
Yds.2
Y/R
TD.2
Fmb
FL
TD.3
2PM
2PP
FantPt
PPR
DKPt
FDPt
VBD
PosRank
OvRank
-9999


In [91]:
pd17 = pd17.drop(columns=['Rk', 'Tm', 'Player', 'FantPos', 'FantPt', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd18 = pd18.drop(columns=['Rk', 'Tm', 'Player', 'FantPos' , 'FantPt', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd19 = pd19.drop(columns=['Rk', 'Tm', 'Player', 'FantPos' , 'FantPt', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd20 = pd20.drop(columns=['Rk', 'Tm', 'Player', 'FantPos' , 'FantPt', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])
pd21 = pd21.drop(columns=['Rk', 'Tm', 'Player', 'FantPos' , 'FantPt', 'DKPt', 'FDPt', 'VBD', 'PosRank', 'OvRank'])

In [92]:
pd17.fillna(0, inplace=True)
pd18.fillna(0, inplace=True)
pd19.fillna(0, inplace=True)
pd20.fillna(0, inplace=True)
pd21.fillna(0, inplace=True)


In [93]:
pd17.head()

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Yds.2,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,-9999
0,23,15,15,0,0,0,0,0,279,1305,...,788,12.31,6,5.0,2,19,0.0,0.0,383.3,GurlTo01
1,25,15,15,0,0,0,0,0,321,1291,...,655,7.71,2,3.0,2,11,0.0,0.0,341.6,BellLe00
2,22,16,16,0,0,0,0,0,272,1327,...,455,8.58,3,1.0,1,11,0.0,0.0,295.2,HuntKa00
3,22,16,3,0,0,0,0,0,120,728,...,826,10.20,5,1.0,1,14,1.0,0.0,320.4,KamaAl00
4,24,16,16,0,0,0,0,0,284,1105,...,476,8.21,4,1.0,0,12,0.0,0.0,288.1,GordMe00


In [94]:
pd18.head()

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Yds.2,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,-9999
0,24,14,14,0,0,0,0,0,256,1251,...,580,9.83,4,1.0,1,21,3.0,0.0,372.1,GurlTo01
1,21,16,16,0,0,0,0,0,261,1307,...,721,7.92,4,0.0,0,15,1.0,0.0,385.8,BarkSa00
2,22,16,16,1,1,50,1,0,219,1098,...,867,8.10,6,4.0,1,13,0.0,0.0,385.5,McCaCh01
3,23,15,13,0,0,0,0,0,194,883,...,709,8.75,4,1.0,0,18,3.0,0.0,354.2,KamaAl00
4,23,16,16,383,580,5097,50,12,60,272,...,0,0.00,0,9.0,2,2,1.0,0.0,417.1,MahoPa00


In [95]:
pd17target = pd18[['PPR','-9999']]

In [96]:
pd17target.head()

,PPR,-9999
0,372.1,GurlTo01
1,385.8,BarkSa00
2,385.5,McCaCh01
3,354.2,KamaAl00
4,417.1,MahoPa00


In [97]:
pd18target = pd19[['PPR','-9999']]
pd19target = pd20[['PPR','-9999']]
pd20target = pd21[['PPR','-9999']]


In [98]:
pd17target.rename(columns={'PPR':'PPRtarget'}, inplace=True)

C:\Users\Peter\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [99]:
pd17target.head()

,PPRtarget,-9999
0,372.1,GurlTo01
1,385.8,BarkSa00
2,385.5,McCaCh01
3,354.2,KamaAl00
4,417.1,MahoPa00


In [100]:
pd18target.rename(columns={'PPR':'PPRtarget'}, inplace=True)
pd19target.rename(columns={'PPR':'PPRtarget'}, inplace=True)
pd20target.rename(columns={'PPR':'PPRtarget'}, inplace=True)

C:\Users\Peter\anaconda3\envs\learn-env\lib\site-packages\pandas\core\frame.py:4296: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  return super().rename(


In [101]:
pd18target.head()

,PPRtarget,-9999
0,471.2,McCaCh01
1,415.7,JackLa00
2,294.6,HenrDe00
3,314.8,JoneAa00
4,311.7,ElliEz00


In [102]:
pd17final = pd17.merge(pd17target,how='left', left_on='-9999', right_on='-9999')

In [103]:
pd17final.head()

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,-9999,PPRtarget
0,23,15,15,0,0,0,0,0,279,1305,...,12.31,6,5.0,2,19,0.0,0.0,383.3,GurlTo01,372.1
1,25,15,15,0,0,0,0,0,321,1291,...,7.71,2,3.0,2,11,0.0,0.0,341.6,BellLe00,NaN
2,22,16,16,0,0,0,0,0,272,1327,...,8.58,3,1.0,1,11,0.0,0.0,295.2,HuntKa00,230.2
3,22,16,3,0,0,0,0,0,120,728,...,10.20,5,1.0,1,14,1.0,0.0,320.4,KamaAl00,354.2
4,24,16,16,0,0,0,0,0,284,1105,...,8.21,4,1.0,0,12,0.0,0.0,288.1,GordMe00,275.5


In [104]:
pd18final = pd18.merge(pd18target,how='left', left_on='-9999', right_on='-9999')
pd19final = pd19.merge(pd19target,how='left', left_on='-9999', right_on='-9999')
pd20final = pd20.merge(pd20target,how='left', left_on='-9999', right_on='-9999')

In [105]:
df_final = pd.DataFrame().append([pd18final,pd19final, pd20final, pd17final])

In [106]:
df_final.head()

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,-9999,PPRtarget
0,24,14,14,0,0,0,0,0,256,1251,...,9.83,4,1.0,1,21,3.0,0.0,372.1,GurlTo01,219.4
1,21,16,16,0,0,0,0,0,261,1307,...,7.92,4,0.0,0,15,1.0,0.0,385.8,BarkSa00,244.1
2,22,16,16,1,1,50,1,0,219,1098,...,8.10,6,4.0,1,13,0.0,0.0,385.5,McCaCh01,471.2
3,23,15,13,0,0,0,0,0,194,883,...,8.75,4,1.0,0,18,3.0,0.0,354.2,KamaAl00,248.5
4,23,16,16,383,580,5097,50,12,60,272,...,0.00,0,9.0,2,2,1.0,0.0,417.1,MahoPa00,287.0


In [107]:
df_final = df_final.sort_values('PPR', ascending=False)

In [108]:
df_final.head(20)

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,-9999,PPRtarget
0,23,16,16,0,2,0,0,0,287,1387,...,8.66,4,1.0,0,19,1.0,0.0,471.2,McCaCh01,90.4
4,23,16,16,383,580,5097,50,12,60,272,...,0.00,0,9.0,2,2,1.0,0.0,417.1,MahoPa00,287.0
1,22,15,15,265,401,3127,36,6,176,1206,...,0.00,0,9.0,2,7,0.0,0.0,415.7,JackLa00,332.8
6,24,16,16,396,572,4544,37,10,102,421,...,12.00,1,9.0,6,9,0.0,0.0,396.1,AlleJo02,402.6
1,21,16,16,0,0,0,0,0,261,1307,...,7.92,4,0.0,0,15,1.0,0.0,385.8,BarkSa00,244.1
2,22,16,16,1,1,50,1,0,219,1098,...,8.10,6,4.0,1,13,0.0,0.0,385.5,McCaCh01,471.2
0,23,15,15,0,0,0,0,0,279,1305,...,12.31,6,5.0,2,19,0.0,0.0,383.3,GurlTo01,372.1
7,37,16,16,372,526,4299,48,5,38,149,...,-6.00,0,4.0,2,3,0.0,0.0,383.3,RodgAa00,333.3
8,23,16,16,375,558,3971,26,12,133,819,...,0.00,0,9.0,4,11,0.0,0.0,378.7,MurrKy00,300.5
1,25,15,10,0,0,0,0,0,187,932,...,9.11,5,1.0,0,21,0.0,0.0,377.8,KamaAl00,234.7


In [109]:
df_final.shape

(2492, 25)

In [110]:
df_final.fillna(0, inplace=True)

In [111]:
df_final = df_final.drop(columns=['-9999'])

In [112]:
df_final=df_final.head(1000)

In [113]:
df_final.shape

(1000, 24)

In [114]:
df_final.head(20)

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Yds.2,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,PPRtarget
0,23,16,16,0,2,0,0,0,287,1387,...,1005,8.66,4,1.0,0,19,1.0,0.0,471.2,90.4
4,23,16,16,383,580,5097,50,12,60,272,...,0,0.00,0,9.0,2,2,1.0,0.0,417.1,287.0
1,22,15,15,265,401,3127,36,6,176,1206,...,0,0.00,0,9.0,2,7,0.0,0.0,415.7,332.8
6,24,16,16,396,572,4544,37,10,102,421,...,12,12.00,1,9.0,6,9,0.0,0.0,396.1,402.6
1,21,16,16,0,0,0,0,0,261,1307,...,721,7.92,4,0.0,0,15,1.0,0.0,385.8,244.1
2,22,16,16,1,1,50,1,0,219,1098,...,867,8.10,6,4.0,1,13,0.0,0.0,385.5,471.2
0,23,15,15,0,0,0,0,0,279,1305,...,788,12.31,6,5.0,2,19,0.0,0.0,383.3,372.1
7,37,16,16,372,526,4299,48,5,38,149,...,-6,-6.00,0,4.0,2,3,0.0,0.0,383.3,333.3
8,23,16,16,375,558,3971,26,12,133,819,...,0,0.00,0,9.0,4,11,0.0,0.0,378.7,300.5
1,25,15,10,0,0,0,0,0,187,932,...,756,9.11,5,1.0,0,21,0.0,0.0,377.8,234.7


In [115]:
df_final.tail()

,Age,G,GS,Cmp,Att,Yds,TD,Int,Att.1,Yds.1,...,Yds.2,Y/R,TD.2,Fmb,FL,TD.3,2PM,2PP,PPR,PPRtarget
262,22,16,1,0,0,0,0,0,63,273,...,109,4.54,0,1.0,1,0,0.0,0.0,60.2,63.3
257,25,16,4,0,0,0,0,0,3,17,...,315,13.70,1,2.0,1,1,0.0,0.0,60.2,37.8
265,30,14,12,0,0,0,0,0,0,0,...,272,10.88,1,0.0,0,1,1.0,0.0,60.2,40.7
252,28,16,0,0,0,0,0,0,0,0,...,271,12.90,2,0.0,0,2,0.0,0.0,60.1,7.1
240,24,16,3,0,0,0,0,0,60,336,...,85,5.31,0,2.0,2,1,0.0,0.0,60.1,54.0


## Dummy Model

In [75]:
##for dummy model insert prediction simply based on last years rankings? ie Jonathon Taylor #1, Cooper Kupp #2 etc

In [116]:
X = df_final.drop(columns=['PPRtarget'])
y = df_final['PPRtarget']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42) 

In [117]:
ss = StandardScaler()
ss.fit(X_train)
X_train_scaled = ss.transform(X_train)
X_test_scaled = ss.transform(X_test)
y_train.value_counts(normalize=True)

0.0      0.09125
240.0    0.00500
87.0     0.00375
75.3     0.00375
156.7    0.00375
          ...   
117.6    0.00125
164.3    0.00125
165.3    0.00125
208.7    0.00125
0.4      0.00125
Name: PPRtarget, Length: 643, dtype: float64

## Dense Neural Network Model

In [125]:
dense_model = models.Sequential()
dense_model.add(layers.Dense(128, activation='relu', input_dim=23))
dense_model.add(layers.Dense(64, activation='relu'))
dense_model.add(layers.Dense(1, activation='sigmoid'))

dense_model.summary()

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 128)               3072      
_________________________________________________________________
dense_7 (Dense)              (None, 64)                8256      
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 65        
Total params: 11,393
Trainable params: 11,393
Non-trainable params: 0
_________________________________________________________________


In [126]:
dense_model.compile(optimizer='adam',
                loss='binary_crossentropy',
                metrics=['accuracy'])

In [128]:
dense_model.fit(X_train_scaled, y_train, epochs=1000, batch_size=128)

Epoch 1/1000
7/7 [==============================] - 0s 2ms/step - loss: -9135520.0000 - accuracy: 0.0000e+00
Epoch 2/1000
7/7 [==============================] - 0s 712us/step - loss: -9411506.0000 - accuracy: 0.0000e+00
Epoch 3/1000
7/7 [==============================] - 0s 712us/step - loss: -9687700.0000 - accuracy: 0.0000e+00
Epoch 4/1000
7/7 [==============================] - 0s 713us/step - loss: -9972831.0000 - accuracy: 0.0000e+00
Epoch 5/1000
7/7 [==============================] - 0s 712us/step - loss: -10254182.0000 - accuracy: 0.0000e+00
Epoch 6/1000
7/7 [==============================] - 0s 691us/step - loss: -10551265.0000 - accuracy: 0.0000e+00
Epoch 7/1000
7/7 [==============================] - 0s 712us/step - loss: -10851049.0000 - accuracy: 0.0000e+00
Epoch 8/1000
7/7 [==============================] - 0s 738us/step - loss: -11165701.0000 - accuracy: 0.0000e+00
Epoch 9/1000
7/7 [==============================] - 0s 855us/step - loss: -11465403.0000 - accuracy: 0.0000e+0

7/7 [==============================] - 0s 632us/step - loss: -44381032.0000 - accuracy: 0.0000e+00
Epoch 74/1000
7/7 [==============================] - 0s 712us/step - loss: -45123876.0000 - accuracy: 0.0000e+00
Epoch 75/1000
7/7 [==============================] - 0s 619us/step - loss: -45845684.0000 - accuracy: 0.0000e+00
Epoch 76/1000
7/7 [==============================] - 0s 739us/step - loss: -46617960.0000 - accuracy: 0.0000e+00
Epoch 77/1000
7/7 [==============================] - 0s 568us/step - loss: -47388172.0000 - accuracy: 0.0000e+00
Epoch 78/1000
7/7 [==============================] - 0s 712us/step - loss: -48173352.0000 - accuracy: 0.0000e+00
Epoch 79/1000
7/7 [==============================] - 0s 712us/step - loss: -48916900.0000 - accuracy: 0.0000e+00
Epoch 80/1000
7/7 [==============================] - 0s 573us/step - loss: -49729968.0000 - accuracy: 0.0000e+00
Epoch 81/1000
7/7 [==============================] - 0s 712us/step - loss: -50515624.0000 - accuracy: 0.0000e+

7/7 [==============================] - 0s 622us/step - loss: -116918208.0000 - accuracy: 0.0000e+00
Epoch 146/1000
7/7 [==============================] - ETA: 0s - loss: -117577536.0000 - accuracy: 0.0000e+0 - 0s 864us/step - loss: -118226840.0000 - accuracy: 0.0000e+00
Epoch 147/1000
7/7 [==============================] - 0s 731us/step - loss: -119529240.0000 - accuracy: 0.0000e+00
Epoch 148/1000
7/7 [==============================] - ETA: 0s - loss: -141382992.0000 - accuracy: 0.0000e+0 - 0s 712us/step - loss: -120870680.0000 - accuracy: 0.0000e+00
Epoch 149/1000
7/7 [==============================] - 0s 570us/step - loss: -122188608.0000 - accuracy: 0.0000e+00
Epoch 150/1000
7/7 [==============================] - 0s 642us/step - loss: -123550344.0000 - accuracy: 0.0000e+00
Epoch 151/1000
7/7 [==============================] - 0s 712us/step - loss: -124870176.0000 - accuracy: 0.0000e+00
Epoch 152/1000
7/7 [==============================] - 0s 671us/step - loss: -126249360.0000 - accu

7/7 [==============================] - 0s 599us/step - loss: -228680112.0000 - accuracy: 0.0000e+00
Epoch 216/1000
7/7 [==============================] - 0s 713us/step - loss: -230579056.0000 - accuracy: 0.0000e+00
Epoch 217/1000
7/7 [==============================] - 0s 712us/step - loss: -232543504.0000 - accuracy: 0.0000e+00
Epoch 218/1000
7/7 [==============================] - 0s 807us/step - loss: -234389184.0000 - accuracy: 0.0000e+00
Epoch 219/1000
7/7 [==============================] - 0s 712us/step - loss: -236355152.0000 - accuracy: 0.0000e+00
Epoch 220/1000
7/7 [==============================] - 0s 537us/step - loss: -238251104.0000 - accuracy: 0.0000e+00
Epoch 221/1000
7/7 [==============================] - 0s 712us/step - loss: -240181312.0000 - accuracy: 0.0000e+00
Epoch 222/1000
7/7 [==============================] - 0s 736us/step - loss: -242132384.0000 - accuracy: 0.0000e+00
Epoch 223/1000
7/7 [==============================] - 0s 616us/step - loss: -244134768.0000 - a

7/7 [==============================] - 0s 712us/step - loss: -388914880.0000 - accuracy: 0.0000e+00
Epoch 287/1000
7/7 [==============================] - 0s 619us/step - loss: -391525952.0000 - accuracy: 0.0000e+00
Epoch 288/1000
7/7 [==============================] - 0s 570us/step - loss: -394110656.0000 - accuracy: 0.0000e+00
Epoch 289/1000
7/7 [==============================] - 0s 744us/step - loss: -396782880.0000 - accuracy: 0.0000e+00
Epoch 290/1000
7/7 [==============================] - 0s 712us/step - loss: -399349792.0000 - accuracy: 0.0000e+00
Epoch 291/1000
7/7 [==============================] - 0s 712us/step - loss: -402003328.0000 - accuracy: 0.0000e+00
Epoch 292/1000
7/7 [==============================] - 0s 718us/step - loss: -404578336.0000 - accuracy: 0.0000e+00
Epoch 293/1000
7/7 [==============================] - 0s 713us/step - loss: -407243296.0000 - accuracy: 0.0000e+00
Epoch 294/1000
7/7 [==============================] - 0s 731us/step - loss: -409843232.0000 - a

...

7/7 [==============================] - 0s 697us/step - loss: -2461313280.0000 - accuracy: 0.0000e+00
Epoch 709/1000
7/7 [==============================] - 0s 712us/step - loss: -2468873472.0000 - accuracy: 0.0000e+00
Epoch 710/1000
7/7 [==============================] - 0s 712us/step - loss: -2476094464.0000 - accuracy: 0.0000e+00
Epoch 711/1000
7/7 [==============================] - 0s 715us/step - loss: -2483551488.0000 - accuracy: 0.0000e+00
Epoch 712/1000
7/7 [==============================] - 0s 570us/step - loss: -2490978048.0000 - accuracy: 0.0000e+00
Epoch 713/1000
7/7 [==============================] - 0s 680us/step - loss: -2498420480.0000 - accuracy: 0.0000e+00
Epoch 714/1000
7/7 [==============================] - 0s 712us/step - loss: -2505846016.0000 - accuracy: 0.0000e+00
Epoch 715/1000
7/7 [==============================] - 0s 591us/step - loss: -2513585152.0000 - accuracy: 0.0000e+00
Epoch 716/1000
7/7 [==============================] - 0s 570us/step - loss: -2521040384

7/7 [==============================] - 0s 570us/step - loss: -3015852032.0000 - accuracy: 0.0000e+00
Epoch 779/1000
7/7 [==============================] - 0s 592us/step - loss: -3024177408.0000 - accuracy: 0.0000e+00
Epoch 780/1000
7/7 [==============================] - 0s 570us/step - loss: -3032416256.0000 - accuracy: 0.0000e+00
Epoch 781/1000
7/7 [==============================] - 0s 809us/step - loss: -3041138176.0000 - accuracy: 0.0000e+00
Epoch 782/1000
7/7 [==============================] - 0s 713us/step - loss: -3049304320.0000 - accuracy: 0.0000e+00
Epoch 783/1000
7/7 [==============================] - 0s 570us/step - loss: -3057999616.0000 - accuracy: 0.0000e+00
Epoch 784/1000
7/7 [==============================] - 0s 642us/step - loss: -3066813440.0000 - accuracy: 0.0000e+00
Epoch 785/1000
7/7 [==============================] - 0s 712us/step - loss: -3075398656.0000 - accuracy: 0.0000e+00
Epoch 786/1000
7/7 [==============================] - 0s 665us/step - loss: -3084175360

7/7 [==============================] - 0s 671us/step - loss: -3637032960.0000 - accuracy: 0.0000e+00
Epoch 849/1000
7/7 [==============================] - 0s 712us/step - loss: -3646768128.0000 - accuracy: 0.0000e+00
Epoch 850/1000
7/7 [==============================] - 0s 754us/step - loss: -3656079104.0000 - accuracy: 0.0000e+00
Epoch 851/1000
7/7 [==============================] - 0s 712us/step - loss: -3665462272.0000 - accuracy: 0.0000e+00
Epoch 852/1000
7/7 [==============================] - 0s 855us/step - loss: -3675212288.0000 - accuracy: 0.0000e+00
Epoch 853/1000
7/7 [==============================] - 0s 642us/step - loss: -3684817920.0000 - accuracy: 0.0000e+00
Epoch 854/1000
7/7 [==============================] - 0s 713us/step - loss: -3694145792.0000 - accuracy: 0.0000e+00
Epoch 855/1000
7/7 [==============================] - 0s 667us/step - loss: -3703890688.0000 - accuracy: 0.0000e+00
Epoch 856/1000
7/7 [==============================] - 0s 712us/step - loss: -3713293056

7/7 [==============================] - 0s 712us/step - loss: -4326942208.0000 - accuracy: 0.0000e+00
Epoch 919/1000
7/7 [==============================] - 0s 712us/step - loss: -4337177600.0000 - accuracy: 0.0000e+00
Epoch 920/1000
7/7 [==============================] - 0s 645us/step - loss: -4347511296.0000 - accuracy: 0.0000e+00
Epoch 921/1000
7/7 [==============================] - 0s 712us/step - loss: -4357993472.0000 - accuracy: 0.0000e+00
Epoch 922/1000
7/7 [==============================] - 0s 676us/step - loss: -4368256000.0000 - accuracy: 0.0000e+00
Epoch 923/1000
7/7 [==============================] - 0s 712us/step - loss: -4378897408.0000 - accuracy: 0.0000e+00
Epoch 924/1000
7/7 [==============================] - 0s 663us/step - loss: -4388870656.0000 - accuracy: 0.0000e+00
Epoch 925/1000
7/7 [==============================] - 0s 712us/step - loss: -4399466496.0000 - accuracy: 0.0000e+00
Epoch 926/1000
7/7 [==============================] - 0s 570us/step - loss: -4409793024

7/7 [==============================] - 0s 570us/step - loss: -5089798656.0000 - accuracy: 0.0000e+00
Epoch 989/1000
7/7 [==============================] - 0s 754us/step - loss: -5101160448.0000 - accuracy: 0.0000e+00
Epoch 990/1000
7/7 [==============================] - 0s 712us/step - loss: -5113061376.0000 - accuracy: 0.0000e+00
Epoch 991/1000
7/7 [==============================] - 0s 735us/step - loss: -5124346880.0000 - accuracy: 0.0000e+00
Epoch 992/1000
7/7 [==============================] - 0s 756us/step - loss: -5135455232.0000 - accuracy: 0.0000e+00
Epoch 993/1000
7/7 [==============================] - 0s 570us/step - loss: -5147236352.0000 - accuracy: 0.0000e+00
Epoch 994/1000
7/7 [==============================] - 0s 695us/step - loss: -5158526464.0000 - accuracy: 0.0000e+00
Epoch 995/1000
7/7 [==============================] - 0s 712us/step - loss: -5170039296.0000 - accuracy: 0.0000e+00
Epoch 996/1000
7/7 [==============================] - 0s 639us/step - loss: -5181800960

## Linear Regression

## Gradient Boosting